In [2]:
import nltk
import numpy as np
import pandas as pd
from nltk import ngrams
from nltk.probability import FreqDist
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ghadyani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:

data = pd.read_csv('google_play_store_apps_reviews.csv')


train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [4]:

def get_ngrams(text, n):
    tokens = nltk.word_tokenize(text)
    return list(ngrams(tokens, n))

def train_ngram(data, n):
    positive_ngrams = []
    negative_ngrams = []

    for index, row in data.iterrows():
        grams = get_ngrams(row['review'], n)
        if row['polarity'] == 1:
            positive_ngrams.extend(grams)
        elif row['polarity'] == 0:
            negative_ngrams.extend(grams)

    positive_freq = FreqDist(positive_ngrams)
    negative_freq = FreqDist(negative_ngrams)

    return positive_freq, negative_freq


n = 2  
positive_freq, negative_freq = train_ngram(train_data, n)

In [6]:
def test_ngram(data, positive_freq, negative_freq, n):
    
    pred_labels = []
    
    for index, row in data.iterrows():
        # tokenize the review into n-grams
        grams = get_ngrams(row['review'], n)
        
        # total probability of positive and negative n-grams
        positive_prob = sum([positive_freq[gram] for gram in grams])
        negative_prob = sum([negative_freq[gram] for gram in grams])
        
        # predict the label based on which probability is higher
        pred_label = 1 if positive_prob > negative_prob else 0
        
     
        pred_labels.append(pred_label)
    
    
    return pred_labels


In [7]:
from sklearn.metrics import accuracy_score


actual_labels = test_data['polarity']


pred_labels = test_ngram(test_data, positive_freq, negative_freq, n)


accuracy = accuracy_score(actual_labels, pred_labels)
print("Accuracy:", accuracy)


Accuracy: 0.7653631284916201
